# <center>ZILLOW FINAL REPORT</center> 
____________________________________________________________

## <span style="color: green;">EXECUTIVE SUMMARY</span>


### PROJECT DESCRIPTION
This project predicts factors contributing to 2017 property tax assessed values for three California counties ( Orange County, Ventura, Los Angeles). The data collected from this project can help with future planning on ways to decrease property tax and improve loyalty to the Zillow brand.

### PROJECT GOALS
* This project goal is to develop an ML Regression model to accurately forecast the assessed values of Single Family Properties by leveraging property attributes. 
* Identify the crucial factors property tax assessed values of single family properties that had a transaction during 2017. 
* Present findings to lead data scientist and other Zillow stakeholders


### INITIAL QUESTIONS
1. Does the Lot Size have a significant impact on property tax value?
2. Is there a relationship between square feet and property tax value?
3. Does number of bedrooms have a relationship to property tax value?
4. Does number of bathrooms have a relationship to property tax value?


## DATA DICTIONARY
| Original Name               |   Target     |       Datatype         |       Definition             |
|:---------------------------:|:------------:|:----------------------:|------------------------------|
|  taxvaluedollarcnt          |  tax_value   | 50283 non-null: float64|  target variable            

|      Original Name          |   Renamed    |       Datatype         |     Definition               |
|-----------------------------|--------------|------------------------|------------------------------|
|  bedroomcnt                 |  bedrooms    | 50283 non-null: int64  | # of bedrooms                |
|  bathroomcnt                |  bathrooms   | 50283 non-null: int64  | # of bathrooms               |
|calculatedfinishedsquarefeet |  sqft        | 50283 non-null: int64  | # of square feet             |
|  yearbuilt                  |  year_built  | 50283 non-null: object | year house was built)        |
|  fips                       |  county      | 50283 non-null: object | County house located         |
|  lotsizesquarefeet          |  lot_size    | 50283 non-null: int64  | size of lot                  |
|  longitude                  |  longitude   | 50283 non-null: int64  | longitude line house located |
|  latitude                   |  latitude    | 50283 non-null: int64  | latitude line house located  |

## <span style="color: green;">AQUISITION AND PREPARATION</span>
- Data was acquired from the Codeup, Inc database using Kaggle's 2017 Zillow data.
- The dataset initially consisted of 9 columns and 52,441 rows before any cleaning procedures were applied.
- Each row in the dataset represents a single-family home property ('261' parcelid).

- The data was prepared by performing various steps. Firstly, the columns were renamed to enhance readability. Null values and duplicate rows were dropped. Data types were modified to facilitate exploration and modeling.

- The county information, labeled as "fips," was renamed appropriately. 

- To address skewed data, outliers were removed. For instance, properties with more than 6 bedrooms or bathrooms and prices exceeding 2M were considered outliers. 
- Resulting in 50,282 rows or %95 of original data used to maintain data integrity
- The data was split into training, validation, and testing sets using a 50%, 30%, 20% split, respectively using Random State of 123.

In [ ]:
#python libraries
import pandas as pd
import numpy as np
import os

#import visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns 

# Import homemade files
from env import hostname, user, password
import wrangle as w
# import explore as e

# Import scikit-learn for split
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

# Import for scaling
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler,QuantileTransformer

# Import for feature selection
from sklearn.feature_selection import SelectKBest, f_regression, RFE

# Import scikit-learn modeling 
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import explained_variance_score

#turn of warnings 
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
df = wrangle.get_zillow_data(df)
df.head()

## <span style="color: green;">EXPLORATORY ANALYSIS</span>

## <span style="color: green;">MODELING</span>

## <span style="color: green;">CONCLUSION</span>

## <span style="color: green;">NEXT STEPS </span>

## <span style="color: green;">RECOMMENDATIONS</span>